In [2]:
from datasets import load_dataset
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas
import openai
from openai import APIError
import os
import json
import re
import numpy as np
from sklearn.cluster import KMeans
from pprint import pprint
from pathlib import Path
import tiktoken


# Load the .env file
load_dotenv()

# Access the variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY
pd.set_option('display.max_colwidth', None)  # None means no limit on column width
pd.set_option('display.width', 300)  

In [3]:
df = pandas.read_parquet('datasets/STS.parquet')


In [8]:
df['score'].describe()

count    1379.000000
mean        2.607917
std         1.525517
min         0.000000
25%         1.315000
50%         2.800000
75%         3.800000
max         5.000000
Name: score, dtype: float64

In [1]:
from srbai import transliterate_cir2lat








ModuleNotFoundError: No module named 'srbai'